<h1>1. Testing Random Environment with OpenAI Gym</h1>

In [262]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [263]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature Array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 - 1 =-1 temperature
        # 1 - 1 = 0
        # 2 - 1 = 1 temperature
        self.state += action - 1
        # Reduce shower length by 1 second
        self.shower_length -=1
        
        # Calculate reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
            
        # Check if shower is done
        if self.shower_length <=0:
            done=True
        else:
            done=False
            
        # Apply temperature noise
        # self.state += random.randint(-1, 1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3, 3)
        # Reset shower time
        self.shower_length = 60
        return self.state

In [264]:
env = ShowerEnv()

/home/neel/anaconda3/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [265]:
env.observation_space.sample()

array([73.318405], dtype=float32)

In [266]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-42
Episode:2 Score:-14
Episode:3 Score:-12
Episode:4 Score:-12
Episode:5 Score:-14
Episode:6 Score:-26
Episode:7 Score:-6
Episode:8 Score:-44
Episode:9 Score:-6
Episode:10 Score:-2


<h1>2. Create a Deep Learning Model with Keras</h1>

In [267]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [268]:
states = env.observation_space.shape
actions = env.action_space.n

In [269]:
actions

3

In [270]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [271]:
# del model

In [272]:
model = build_model(states, actions)

In [273]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 24)                48        
                                                                 
 dense_55 (Dense)            (None, 24)                600       
                                                                 
 dense_56 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


<h1>3. Build Agent with Keras-RL</h1>

In [274]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [275]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup = 10, target_model_update=1e-2)
    return dqn

In [276]:
import tensorflow as tf
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2023-05-01 20:37:58.946445: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_55_1/bias/Assign' id:3908 op device:{requested: '', assigned: ''} def:{{{node dense_55_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_55_1/bias, dense_55_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 28:38 - reward: -1.0000

/home/neel/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-05-01 20:37:59.179341: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_56/BiasAdd' id:3865 op device:{requested: '', assigned: ''} def:{{{node dense_56/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_56/MatMul, dense_56/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-01 20:37:59.252443: W tensorflow/c/c_api.cc:300] Operation '{name:'count_13/Assign' id:4081 op device:{requested: '', assigned: ''} def:{{{node count_13/Assign}} = AssignVariableOp[_has_manual_contro

10000/10000 [==============================] - 50s 5ms/step - reward: -0.6512
166 episodes - episode_reward: -39.277 [-60.000, 42.000] - loss: 1.135 - mae: 7.098 - mean_q: -9.795

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: -0.5026
167 episodes - episode_reward: -29.928 [-60.000, 40.000] - loss: 2.588 - mae: 11.967 - mean_q: -17.231

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: -0.4940
167 episodes - episode_reward: -29.701 [-60.000, 52.000] - loss: 2.613 - mae: 11.793 - mean_q: -16.965

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: -0.4178
166 episodes - episode_reward: -25.048 [-60.000, 40.000] - loss: 2.524 - mae: 11.568 - mean_q: -16.651

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: -0.4958
done, took 266.879 seconds


In [277]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -58.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -54.000, steps: 60
Episode 4: reward: -54.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -54.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -54.000, steps: 60
Episode 11: reward: -54.000, steps: 60
Episode 12: reward: -54.000, steps: 60
Episode 13: reward: -56.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -54.000, steps: 60
Episode 16: reward: -54.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: -58.000, steps: 60
Episode 19: reward: -56.000, steps: 60
Episode 20: reward: -56.000, steps: 60
Episode 21: reward: -54.000, steps: 60
Episode 22: reward: -54.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: -60.000, steps: 60
Episode 25: reward: -58.000, steps: 60
Episo

In [278]:
# _ = dqn.test(env, nb_episodes=15, visualize=True)

<h1>4. Reloading Agent from Memory</h1>

In [279]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [280]:
del model
del dqn
del env

In [283]:
import gym
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])

2023-05-01 20:49:38.847749: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_61/bias/Assign' id:5020 op device:{requested: '', assigned: ''} def:{{{node dense_61/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_61/bias, dense_61/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [284]:
dqn.load_weights('dqn_weights.h5f')

2023-05-01 20:49:41.763169: W tensorflow/c/c_api.cc:300] Operation '{name:'total_21/Assign' id:5260 op device:{requested: '', assigned: ''} def:{{{node total_21/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_21, total_21/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


ValueError: Layer weight shape (4, 24) not compatible with provided weight shape (1, 24)

In [285]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


ValueError: Error when checking input: expected dense_60_input to have 2 dimensions, but got array with shape (1, 1, 2)